# Love Behavior with Human Following

**Step 1: Create display widgets**

In [1]:
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

# Create widgets for displaying images
display_color = widgets.Image(format='jpeg', width='60%')
display_depth = widgets.Image(format='jpeg', width='60%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# Convert numpy array to jpeg coded data for displaying
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

**Step 2: Load YOLO model for human detection**

In [2]:
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("yolo11l_half.engine")

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


**Step 3: Initialize camera and implement love behavior with human following**

The robot will:
- Follow detected human by adjusting orientation (turn left/right)
- Approach to maintain optimal distance (450-600mm)
- Stop when human is at comfortable distance
- Only follow humans, not all objects

In [3]:
import traitlets
import numpy as np
import pyzed.sl as sl
import threading
import motors
from traitlets.config.configurable import SingletonConfigurable

# Initialize the Robot class
robot = motors.MotorsYukon(mecanum=False)

class Camera(SingletonConfigurable):
    color_value = traitlets.Any()  # Monitor color_value variable
    
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER

        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

        self.runtime = sl.RuntimeParameters()
        self.thread_runnning_flag = False

        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)

    def _capture_frames(self):
        while(self.thread_runnning_flag == True):
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value_BGRA = self.image.get_data()
                self.color_value = cv2.cvtColor(self.color_value_BGRA, cv2.COLOR_BGRA2BGR)
                self.depth_image = np.asanyarray(self.depth.get_data())
                
    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join()
            robot.stop()

camera = Camera()
camera.start()

[2025-12-05 10:21:18 UTC][ZED][INFO] Logging level INFO
,[2025-12-05 10:21:18 UTC][ZED][INFO] Logging level INFO
,[2025-12-05 10:21:18 UTC][ZED][INFO] Logging level INFO
,[2025-12-05 10:21:19 UTC][ZED][INFO] [Init]  Depth mode: ULTRA
,[2025-12-05 10:21:20 UTC][ZED][INFO] [Init]  Camera successfully opened.
,[2025-12-05 10:21:20 UTC][ZED][INFO] [Init]  Camera FW version: 1523
,[2025-12-05 10:21:20 UTC][ZED][INFO] [Init]  Video mode: VGA@100
,[2025-12-05 10:21:20 UTC][ZED][INFO] [Init]  Serial Number: S/N 36955685
,Loading yolo11l_half.engine for TensorRT inference...
,Love: No human detected - waitingp

**Step 4: Implement love behavior with human following callback**

In [4]:
# Step 4: Enhanced person-following with obstacle avoidance and occlusion handling

def enhanced_follow_callback(change):
    frame = change['new']
    
    # Run YOLO inference to detect humans
    results = model(frame, verbose=False)
    
    # Prepare depth colormap for display
    depth_colormap = cv2.applyColorMap(
        cv2.convertScaleAbs(camera.depth_image, alpha=0.03), 
        cv2.COLORMAP_JET)
    
    conf_threshold = 0.5
    human_detected = False
    closest_distance = float('inf')
    closest_bbox = None
    all_humans = []  # Track all detected humans for occlusion handling
    
    # Detect all humans in frame
    for result in results:
        for i in range(len(result.boxes.cls)):
            if result.boxes.cls[i] == 0:  # Human subject
                if result.boxes.conf[i] > conf_threshold:
                    bbox = result.boxes.xyxy[i]
                    
                    # Calculate center of bounding box
                    center_x = int((bbox[0] + bbox[2]) / 2)
                    center_y = int((bbox[1] + bbox[3]) / 2)
                    
                    # Get depth at human center
                    if (0 <= center_y < camera.height and 
                        0 <= center_x < camera.width):
                        distance = camera.depth_image[center_y, center_x]
                        
                        if not np.isnan(distance) and distance > 0:
                            all_humans.append({
                                'bbox': bbox,
                                'distance': distance,
                                'center_x': center_x,
                                'center_y': center_y
                            })
                            
                            if distance < closest_distance:
                                human_detected = True
                                closest_distance = distance
                                closest_bbox = bbox
    
    # Draw all detected humans
    for human in all_humans:
        bbox = human['bbox']
        # Pink for target (closest), yellow for others
        color = (255, 105, 180) if bbox is closest_bbox else (0, 255, 255)
        cv2.rectangle(frame, 
                     (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), 
                     color, 2)
    
    # COLLISION AVOIDANCE using depth-based obstacle detection
    # (Based on Tutorial 3 Part B collision avoidance)
    depth_image_obstacles = camera.depth_image.copy()
    depth_image_obstacles = np.nan_to_num(depth_image_obstacles, nan=0.0).astype(np.float32)
    
    # Define central collision detection area (from Tutorial 3)
    depth_image_obstacles[:94, :] = 0
    depth_image_obstacles[282:, :] = 0
    depth_image_obstacles[:, :168] = 0
    depth_image_obstacles[:, 504:] = 0
    
    # Filter depth values for obstacle detection
    depth_image_obstacles[depth_image_obstacles < 100] = 0
    depth_image_obstacles[depth_image_obstacles > 2000] = 0
    
    # Check for obstacles in path
    obstacle_detected = False
    obstacle_distance = float('inf')
    if depth_image_obstacles.max() > 0:
        obstacle_distance = depth_image_obstacles[depth_image_obstacles != 0].min()
        if obstacle_distance < 400:  # Obstacle within collision threshold
            obstacle_detected = True
    
    # ENHANCED FOLLOWING BEHAVIOR with obstacle avoidance
    if human_detected and closest_distance != float('inf'):
        # Calculate human position relative to frame center
        bbox_center_x = (closest_bbox[0] + closest_bbox[2]) / 2
        frame_center_x = camera.width / 2
        
        # Calculate horizontal offset from center
        offset_x = bbox_center_x - frame_center_x
        offset_threshold = camera.width * 0.15  # 15% of frame width
        
        # Determine if human needs recentering
        needs_left_turn = offset_x < -offset_threshold
        needs_right_turn = offset_x > offset_threshold
        centered = not needs_left_turn and not needs_right_turn
        
        # PRIORITY 1: Emergency collision avoidance
        if obstacle_detected and obstacle_distance < 300:
            robot.backward(0.3)
            cv2.putText(depth_colormap, f'COLLISION AVOID! {obstacle_distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.putText(depth_colormap, 'RETREATING', 
                       (200, 208), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
            print(f'EMERGENCY: Obstacle at {obstacle_distance:.0f}mm - retreating', end='\r')
        
        # PRIORITY 2: Cautious approach with obstacle awareness
        elif obstacle_detected and obstacle_distance < 500:
            # Obstacle close but not emergency - slow down and navigate around
            if needs_left_turn:
                robot.spinLeft(0.2)
                cv2.putText(depth_colormap, f'CAUTION: Navigate left', 
                           (160, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 165, 255), 2, cv2.LINE_AA)
            elif needs_right_turn:
                robot.spinRight(0.2)
                cv2.putText(depth_colormap, f'CAUTION: Navigate right', 
                           (160, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 165, 255), 2, cv2.LINE_AA)
            else:
                robot.forward(0.15)  # Very slow forward if centered
                cv2.putText(depth_colormap, f'CAUTION: Slow approach', 
                           (160, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 165, 255), 2, cv2.LINE_AA)
            print(f'Caution: Obstacle {obstacle_distance:.0f}mm, Human {closest_distance:.0f}mm', end='\r')
        
        # PRIORITY 3: Normal following behavior (no immediate obstacles)
        elif closest_distance < 450:
            # Too close to target person - retreat
            robot.backward(0.2)
            cv2.putText(depth_colormap, f'Too close {closest_distance:.0f}mm', 
                       (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2, cv2.LINE_AA)
            print(f'Love: Too close {closest_distance:.0f}mm - backing up', end='\r')
            
        elif closest_distance >= 450 and closest_distance <= 600:
            # Optimal distance - maintain position
            robot.stop()
            cv2.putText(depth_colormap, f'Perfect {closest_distance:.0f}mm <3', 
                       (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2, cv2.LINE_AA)
            print(f'Love: Perfect distance {closest_distance:.0f}mm', end='\r')
            
        elif closest_distance > 600 and closest_distance <= 1200:
            # Medium distance - approach while tracking
            if centered:
                robot.forward(0.3)
                cv2.putText(depth_colormap, f'Following {closest_distance:.0f}mm', 
                           (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2, cv2.LINE_AA)
            elif needs_left_turn:
                robot.spinLeft(0.3)
                cv2.putText(depth_colormap, f'Tracking left {closest_distance:.0f}mm', 
                           (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2, cv2.LINE_AA)
            else:  # needs_right_turn
                robot.spinRight(0.3)
                cv2.putText(depth_colormap, f'Tracking right {closest_distance:.0f}mm', 
                           (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2, cv2.LINE_AA)
            print(f'Love: Following {closest_distance:.0f}mm', end='\r')
                
        else:  # closest_distance > 1200
            # Far away - approach faster
            if centered:
                robot.forward(0.5)
                cv2.putText(depth_colormap, f'Approaching {closest_distance:.0f}mm', 
                           (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2, cv2.LINE_AA)
            elif needs_left_turn:
                robot.spinLeft(0.4)
                cv2.putText(depth_colormap, f'Searching left {closest_distance:.0f}mm', 
                           (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2, cv2.LINE_AA)
            else:  # needs_right_turn
                robot.spinRight(0.4)
                cv2.putText(depth_colormap, f'Searching right {closest_distance:.0f}mm', 
                           (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2, cv2.LINE_AA)
            print(f'Love: Fast approach {closest_distance:.0f}mm', end='\r')
        
        # Display occlusion info if multiple humans detected
        if len(all_humans) > 1:
            cv2.putText(frame, f'{len(all_humans)} people detected', 
                       (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                
    else:
        # No human detected - stop and wait (occlusion or person left)
        robot.stop()
        if len(all_humans) == 0:
            cv2.putText(depth_colormap, 'Waiting for person...', 
                       (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2, cv2.LINE_AA)
            print('Love: No person detected - waiting', end='\r')
        else:
            cv2.putText(depth_colormap, 'Target occluded - waiting...', 
                       (160, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2, cv2.LINE_AA)
            print('Love: Target occluded by other people', end='\r')
    
    # Visualize collision detection zone on depth map
    cv2.rectangle(depth_colormap, (168, 94), (504, 282), (0, 255, 0), 1)
    
    # Display obstacle warning if detected
    if obstacle_detected:
        cv2.circle(depth_colormap, (336, 188), 30, (0, 0, 255), 3)
        cv2.putText(depth_colormap, f'OBS: {obstacle_distance:.0f}mm', 
                   (260, 220), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
    
    # Display images
    scale = 0.8
    resized_color = cv2.resize(frame, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    resized_depth = cv2.resize(depth_colormap, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    display_color.value = bgr8_to_jpeg(resized_color)
    display_depth.value = bgr8_to_jpeg(resized_depth)

# Attach callback to camera
camera.observe(enhanced_follow_callback, names=['color_value'])

[12/05/2025-10:21:22] [TRT] [I] Loaded engine size: 52 MiB
,[12/05/2025-10:21:22] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.
,[12/05/2025-10:21:22] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +1, GPU +36, now: CPU 1, GPU 84 (MiB)


**To stop the camera and robot:**

In [ ]:
camera.stop()

**Behavior Description:**

The robot exhibits love behavior by following detected humans:

1. **Distance Measurement**: Calculates distance to closest detected human using depth image
2. **Intelligent Tracking**: 
   - Calculates human position relative to frame center
   - Only turns when human is >15% off-center (avoids continuous adjustment)
   - Turns left/right to recenter human in view
3. **Distance-Based Approach**:
   - **< 450mm**: Slight backward retreat (0.2 speed)
   - **450-600mm**: Stop - optimal "loving" distance
   - **600-1200mm**: Slow approach (0.3) or turn to track (0.3)
   - **> 1200mm**: Fast approach (0.5) or turn to track (0.4)
4. **Smart Following**: 
   - When centered: moves forward toward human
   - When off-center: rotates to recenter before/while approaching
   - When lost: stops and waits (no continuous searching)

The robot follows humans by maintaining visual tracking and optimal proximity, turning only when necessary to keep the human centered in its field of view.